In [1]:
import pandas as pd 

In [2]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [14]:
aggregate = [
  {
    "$match": {
      "session.session_type": "BCI single neuron stim",
      "data_description.data_level": "derived",
      "processing.processing_pipeline.data_processes.start_date_time":{"$gte":"2025-05-09"}
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "session_time": "$session.session_start_time", 
      "session_type": "$session.session_type", 
      "genotype": "$subject.genotype", 
      "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
      "session_number": {
        "$filter": {
          "input": "$session.stimulus_epochs",
          "as": "epoch",
          "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
        }
      },
      'ophys_fov': {
            '$map': {
                'input': '$session.data_streams',
                'as': 'stream',
                'in': {
                    '$map': {
                            'input': '$$stream.ophys_fovs',
                            'as': 'fov',
                            'in': '$$fov.notes'
                    }
                }
            }
        }

    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": 1,
      "session_time": 1, 
      "session_type": 1,
      "genotype": 1,
      "virus_injection_materials": 1,
      "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] },
      "ophys_fov": 1,
    }
  },
  {'$unwind': {'path': '$ophys_fov', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$ophys_fov', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': False}}, 
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': False}}
  
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)


In [15]:
bci_df = pd.DataFrame(records)
bci_df.drop_duplicates(inplace=True)

,_id,name,subject_id,session_time,session_type,genotype,virus_injection_materials,ophys_fov,session_number
0,da9c7414-1bf5-4a0d-a796-3e58cc8d5d3e,single-plane-ophys_754303_2025-01-23_19-48-32_...,754303,2025-01-23T18:36:30.521000-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,3.0
5,db30abf5-90a9-4e3f-a927-a0a9657ba5b1,single-plane-ophys_754303_2025-01-22_18-22-38_...,754303,2025-01-22T16:59:35.618000-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,2.0
10,0fd64eee-aa51-42e0-b37b-32ef53a5263c,single-plane-ophys_766719_2025-01-22_16-45-18_...,766719,2025-01-22T15:36:39.563000-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,2.0
15,c0105605-b166-4db7-bcaf-e7a756d62354,single-plane-ophys_754303_2025-01-31_15-13-50_...,754303,2025-01-31T15:13:50.595999-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,8.0
20,0b6a3ab2-c50c-429a-9588-1e6992ded444,single-plane-ophys_731015_2025-01-28_18-56-35_...,731015,2025-01-28T17:40:57.996000-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,FOV_04,22.0
26,e19d57eb-048c-49fb-a564-9f23b3205c3d,single-plane-ophys_731015_2025-01-24_20-00-44_...,731015,2025-01-24T18:41:22.550000-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,FOV_04,20.0
31,7fe2130f-bfaa-4d5c-8f42-70ea132c0e2a,single-plane-ophys_772414_2025-01-23_16-03-51_...,772414,2025-01-23T14:40:12.962999-08:00,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,3.0
36,d593d409-dff3-4381-be2a-9d3290bfe355,single-plane-ophys_766719_2025-01-27_18-25-39_...,766719,2025-01-27T17:02:50.868000-08:00,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,FOV_01,5.0
41,57178d7b-5c87-4376-933a-769eeabcdd19,single-plane-ophys_731015_2025-01-31_20-37-19_...,731015,2025-01-31T20:37:19.623000-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,FOV_04,23.0
47,5c307543-bf11-4a82-bcdf-85f0886d41cb,single-plane-ophys_731015_2025-01-28_17-40-57_...,731015,2025-01-28T17:40:57.996000-08:00,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,FOV_04,22.0


In [16]:
bci_df.to_csv('/data/metadata.csv', index= False)